In [9]:
%reload_kedro

[05/27/24 12:32:39] INFO     Resolved project path as:                                              ]8;id=323508;file://C:\Users\emili\anaconda3\envs\microsoft_cve_support_venv\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=505896;file://C:\Users\emili\anaconda3\envs\microsoft_cve_support_venv\lib\site-packages\kedro\ipython\__init__.py#139\139]8;;\
                             C:\Users\emili\PycharmProjects\microsoft_cve_support_report.                          
                             To set a different path, run '%reload_kedro <project_root>'                           

[05/27/24 12:32:41] INFO     Kedro project kedro workbench                                          ]8;id=859389;file://C:\Users\emili\anaconda3\envs\microsoft_cve_support_venv\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=239573;file://C:\Users\emili\anaconda3\envs\microsoft_cve_support_venv\lib\site-packages\kedro\ipython\__init__.py#108\108]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=727032;file://C:\Users\emili\anaconda3\envs\microsoft_cve_support_venv\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=448018;file://C:\Users\emili\anaconda3\envs\microsoft_cve_support_venv\lib\site-packages\kedro\ipython\__init__.py#109\109]8;;\
                             'pipelines'                                                                           

In [10]:
from pathlib import Path
# Path.cwd()
# %cd ..
Path.cwd()

WindowsPath('C:/Users/emili/PycharmProjects/microsoft_cve_support_report')

In [11]:
from kedro_workbench.extras.datasets.MongoDataset import MongoDBDocs
# from kedro_workbench.utils.llm_utils import (LLMParams, call_llm_completion_with_logging)
from kedro_workbench.utils.llm_utils import (LLMParams)
from kedro_workbench.utils.athina_logging_utils import (AthinaParams)
import spacy
from kedro.io import DataCatalog
from kedro.config import ConfigLoader
from kedro.framework.project import settings
from kedro.framework.session import KedroSession
from kedro.framework.startup import bootstrap_project
from pathlib import Path

from athina_logger.api_key import AthinaApiKey
# from athina_logger.openai_wrapper import openai
from athina_logger.inference_logger import InferenceLogger
from athina_logger.exception.custom_exception import CustomException
from athina_logger.log_stream_inference.openai_completion_stream import LogOpenAiCompletionStreamInference
from athina_logger.athina_meta import AthinaMeta
from typing import List, Dict, Optional, Any

In [12]:
project_path = Path.cwd()
bootstrap_project(project_path)
session = KedroSession.create(project_path=project_path)
context = session.load_context()

# conf_path = str(context.project_path / settings.CONF_SOURCE)
# conf_loader = ConfigLoader(conf_source=conf_path, env="local")
conf_loader = ConfigLoader(settings.CONF_SOURCE)
conf_catalog = conf_loader["catalog"]
conf_params = conf_loader["parameters"]

credentials = conf_loader["credentials"]
mongo_atlas = credentials["mongo_atlas"]
username = mongo_atlas["username"]
password = mongo_atlas["password"]
openai_api_key = credentials["OPENAI"]["api_key"]

athina_credentials = credentials["athina"]
athina_api_key = athina_credentials['api_key']
AthinaApiKey.set_api_key(athina_api_key)

In [ ]:
def call_llm_completion_with_logging_old(model, system_prompt, input_prompt, max_tokens, temperature, prompt_slug, external_reference_id, cost, custom_attributes):

    client = OpenAI(api_key=openai_api_key)
    response_msg = ""
    payload = {
        "model": model,
        "messages": [
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": input_prompt
            }
        ],
        "max_tokens": max_tokens,
        "temperature": temperature
    }

    try:
        response = client.chat.completions.create(**payload)
        choice = response.choices[0]
        response_msg = choice.message.content
        response_dict = response.model_dump()  # For openai > 1 version

        try:
            InferenceLogger.log_inference(
                prompt_slug=prompt_slug,
                prompt=payload["messages"],
                language_model_id=model,
                response=response_dict,
                external_reference_id=external_reference_id,
                cost=cost,
                custom_attributes=custom_attributes
            )
        except Exception as e:
            if isinstance(e, CustomException):
                print(e.status_code)
                print(e.message)
            else:
                print(e)

    except HTTPError as e:
        print(f"HTTPError occurred: {e}")
        print(f"Max tokens: {max_tokens}\nData sent: system_prompt: {system_prompt}, \ninput_prompt: {input_prompt}")
    except Exception as e:
        print(f"An exception occurred: {e}")

    return response_msg

In [ ]:
model = 'gpt-4o'
system_prompt = 'You are an expert in Microsoft cloud architecture and patching on Azure, Intune, Windows Desktop and mobile devices'
input_prompt = 'What are some common Information Disclosure Vulnerabilities in Microsoft Edge Chromium that system administrators should be on the look out for?'
max_tokens = 750
temperature = 0.80
prompt_slug = 'basic_prompt_slug_v1'
external_reference_id = 'ext_ref_id_1000'

custom_attributes = {
    'environment': 'notebook',
    'experiment': 'athina integration',
    'product_owner': 'Emilio',
    'product_owner_email': 'emilio.gagliardi@portalfuse.io'
    
}

In [ ]:
response = call_llm_completion_with_logging(
    model,
    system_prompt,
    input_prompt,
    max_tokens,
    temperature,
    prompt_slug,
    external_reference_id,
    cost,
    custom_attributes
)
print(response)

In [34]:
from openai import OpenAI
import time
import openai
def call_llm_completion_with_logging(
    llm_params: LLMParams,
    athina_params: AthinaParams
):

    client = OpenAI(api_key=openai_api_key)
    response_msg = ""
    payload = {
        "model": llm_params.model,
        "messages": [
            {
                "role": "system",
                "content": llm_params.system_prompt
            },
            {
                "role": "user",
                "content": llm_params.input_prompt
            }
        ],
        "max_tokens": llm_params.max_tokens,
        "temperature": llm_params.temperature
    }

    try:
        start_time = time.time()
        response = client.chat.completions.create(**payload)
        # response = openai.completions.create(**payload)
        end_time = time.time()
        response_time_ms = (end_time - start_time) * 1000
        choice = response.choices[0]
        response_msg = choice.message.content
        response_dict = response.model_dump()

        log_params = athina_params.__dict__
        log_params.update({
            "prompt": payload["messages"],
            "user_query": llm_params.input_prompt,
            "response": response_dict,
            "language_model_id": llm_params.model,
            "response_time": response_time_ms
        })
        # print(f'log params:\n{log_params}\n')
        try:
            InferenceLogger.log_inference(**log_params)
        except Exception as e:
            if isinstance(e, CustomException):
                print(e.status_code)
                print(e.message)
            else:
                print(e)

    except HTTPError as e:
        print(f"HTTPError occurred: {e}")
        print(f"Max tokens: {llm_params.max_tokens}\n"
              f"Data sent: system_prompt: {llm_params.system_prompt},\n"
              f"input_prompt: {llm_params.input_prompt}"
              )
    except Exception as e:
        print(f"An exception occurred: {e}")
    return response_msg

In [35]:
model = 'gpt-4o'
system_prompt = 'You are an expert in Microsoft cloud architecture and patching on Azure, Intune, Windows Desktop and mobile devices'
input_prompt = 'What are the top 3 most severe Information Disclosure Vulnerabilities in Microsoft Edge Chromium that system administrators should be on the look out for?'
max_tokens = 350
temperature = 0.80
custom_attributes = {
    'experiment': 'athina integration',
    'product_owner': 'Emilio',
    'product_owner_email': 'emilio.gagliardi@portalfuse.io'
    
}

llm_params = LLMParams(
    model=model,
    system_prompt=system_prompt,
    input_prompt=input_prompt,
    max_tokens=max_tokens,
    temperature=temperature
)

athina_params = AthinaParams(
    language_model_id=model,
    prompt=[{"role": "system", "content": system_prompt}, {"role": "user", "content": input_prompt}],
    user_query=None,
    context=None,
    prompt_slug="top_3_information_disclosure_vulnerabilities",
    environment="uplyft_dsm_1_notebook",
    customer_id="PortalFuse",
    customer_user_id="emilio_gagliardi",
    session_id="cve_report_28_05_2024",
    custom_attributes=custom_attributes,
)



In [36]:
# response = call_llm_completion_with_logging(
#     llm_params=llm_params,
#     athina_eval=athina_eval,
#     athina_meta=athina_meta
# )
response = call_llm_completion_with_logging(
    llm_params=llm_params,
    athina_params=athina_params
)
print(response)

System administrators should be vigilant about several severe information disclosure vulnerabilities in Microsoft Edge Chromium. While the specific vulnerabilities can change over time as new ones are discovered and patched, historically, some of the most severe information disclosure vulnerabilities have included:

### 1. **CVE-2021-21148: Type Confusion in V8**
   - **Description**: This vulnerability is due to a type confusion error in the V8 engine, which is used for processing JavaScript in Chromium-based browsers like Microsoft Edge. If exploited, this vulnerability could allow an attacker to execute arbitrary code in the context of the browser, potentially leading to information disclosure.
   - **Impact**: High. An attacker could gain access to sensitive information or take over the user’s session.

### 2. **CVE-2021-21193: Use-After-Free in WebRTC**
   - **Description**: This vulnerability arises from a use-after-free error in WebRTC, the component responsible for real-time co

In [ ]:
response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"}
    ]
)
print(response.choices[0].message["content"])

In [ ]:
"```json\n{\n    \"metadata\": {\n        \"id\": \"6d724387-c7c1-7b37-1f02-6135ac9f0bbf\",\n        \"post_id\": \"CVE-2024-30054\",\n        \"revision\": \"1.0\",\n        \"published\": \"14-05-2024\"\n    },\n    \"classification\": \"Solution provided\"\n}\n```"